# SANDOZ - WEEK 1/2

### 1. Leer PDFs en Python

Usando los 3 pdfs entregados.

  1.1. Método 1:

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.5 MB/s eta 0:00:00


In [ ]:
import PyPDF2

with open("Introduction to Pre-Selection Pipeline.pdf", "rb") as f:
    reader = PyPDF2.PdfReader(f)
    texto = ""
    for pagina in reader.pages:
        texto += pagina.extract_text()


1.2. Método 2:

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.7 MB/s eta 0:00:00


In [ ]:
import pdfplumber

texto = ""
with pdfplumber.open("Introduction to Pre-Selection Pipeline.pdf") as pdf:
    for pagina in pdf.pages:
        texto += pagina.extract_text()


## 2. Llamadas a la API de OpenAI

In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.108.0
    Uninstalling openai-1.108.0:
      Successfully uninstalled openai-1.108.0


In [2]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
import openai

openai.api_key = "poner la clave aqui"

preguntas = [
    "Hola",
    "¿Cuál es la capital de Francia?",
    "¿Cuánto es 5+7?",
    "¿Quién escribió Don Quijote?",
    "¿Cuál es el río más largo del mundo?"
]

for i in range(len(preguntas)):
  response = openai.chat.completions.create(
      model="gpt-4",
      messages=[
          {"role": "user", "content": preguntas[i]}
      ]

  )
  print(response.choices[0].message.content)


¡Hola! ¿Cómo puedo ayudarte hoy?
La capital de Francia es París.
El resultado de sumar 5 y 7 es 12.
Don Quijote fue escrito por Miguel de Cervantes Saavedra.
El río más largo del mundo es el río Nilo en África.


## 3. RAGs

Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response. Large Language Models (LLMs) are trained on vast volumes of data and use billions of parameters to generate original output for tasks like answering questions, translating languages, and completing sentences. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model.

Source: https://aws.amazon.com/what-is/retrieval-augmented-generation/

In [ ]:
!pip install pdfplumber sentence-transformers transformers accelerate

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 45.8 MB/s eta 0:00:00


In [ ]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


# 1. leer pdfs
pdfs = ["CCO relevant Abbreviations & Definitions.pdf", "Introduction to Pre-Selection Pipeline.pdf", "Understanding IP Information.pdf"]

documentos = []
for pdf_path in pdfs:
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            texto = pagina.extract_text()
            if texto:
                documentos.append(texto)

print(f"Se extrajeron {len(documentos)} fragmentos de texto.")


# 2. embedding

emb_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = emb_model.encode(documentos, convert_to_numpy=True)

# Base vectorial en FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


# 3. modelo (ej. flan-t5-base)

modelo = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(modelo)
llm = AutoModelForSeq2SeqLM.from_pretrained(modelo, device_map="auto")

def responder(pregunta, k=3):
    # Buscar fragmentos relevantes
    query_emb = emb_model.encode([pregunta], convert_to_numpy=True)
    D, I = index.search(query_emb, k)

    contexto = "\n".join([documentos[i] for i in I[0]])

    prompt = f"""
    Responde la siguiente pregunta usando el contexto:

    Contexto:
    {contexto}

    Pregunta:
    {pregunta}
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = llm.generate(**inputs, max_new_tokens=300)
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta



Se extrajeron 31 fragmentos de texto.


No funciona bien con abreviaturas.

In [ ]:
print(responder("What is the expansion of ANDA?"))

Abbreviation Expansion ANDA Abbreviated New Drug Application ANDS Abbreviated New Drug Submission ANOVA Analysis of Variance API Active Pharmaceutical Ingredient APO Advanced Planning and Optimization AS&T Analytical Science & Technology (QC lab) AW Artwork B2B Business-to-Business B2C Business-to-Customer BC Business Case BCS Biopharmaceutics Classification System BD Business Development BD&L Business Development & Licensing BE Bioequivalence BE study Bioequivalence Study BoD Board of Directors Abbreviation Expansion SDC Sandoz Development Center SDZ Sandoz SGD Sandoz Global Development SIR Sandoz International Region SKU Stock Keeping Unit SLT Sandoz Leadership Team SPC Supply Point Committee specs Specification SPOC Single Point of Contact SRM Site review meeting SRT Supplier Relationship Team STO Sandoz Technical Operations TechOps Technical Operations TPC Total Product Cost TRA Technical Risk Assessment TW Track Wise US United States of America VMS Vendor Management System WHO Wor

Contesta al resto de preguntas aparentemente bien.

In [ ]:
print(responder("Who is Harus?"))

Harus is a pharmaceutical executive who is responsible for the development of new drugs.
